# 이미지 식별 머신을 위한 데이터를 준비한다.

## 필요한 라이브러리를 불러 온다.

In [5]:
# 데이터 플로팅 라이브러리
import matplotlib.pyplot as plt
# 숫자 처리 라이브러리
import numpy as np
# 딥러닝을 위한 파이토치 라이브러리
import torch
from torch import nn, optim
# 토치비전 라이브러리
import torchvision
from torchvision import datasets, transforms, models
# 이미지 처리 라이브러리 (PIL, pillow)
from PIL import Image
# 주피터 노트북에서 plot이 보이도록 설정
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

## 데이터 디렉토리, 분할 비율, 변환 방법을 설정한다.

In [4]:
# 이미지 데이터가 있는 디렉토리와 데이터 세트 분할 비율(valid_size)을 정한다.



# 이미지 데이터를 ResNet50에서 다룰 수 있도록 변환시키는 방법을 정한다. (t_transforms)





### (확인) 변환 방법을 출력하여 확인해 본다.

In [ ]:
# 설정한 이미지 데이터 변환 방법을 출력하여 확인한다.
print(t_transforms)

## 데이터 로더 함수를 작성한다.

### (연습) trainloader와 testloader를 만들어 본다.

#### 1. 학습 데이터 세트 및 테스트 데이터 세트의 디렉토리 및 변환 방식을 지정한다.

In [ ]:
# datasets.ImageFolder를 사용해서 학습 데이터(train_data)와 테스트 데이터(test_data)를 만든다.




# 학습 데이터의 형식을 확인한다.
print(train_data)

# 학습 데이터와 테스트 데이터의 길이를 확인한다.
print(len(train_data), len(test_data))

#### 2. 데이터세트를 섞기 위해, 우선 인덱스를 만들어 랜덤하게 섞는다.

In [ ]:
# train_data 사이즈만큼의 정수값을 갖는 인덱스 리스트(indices)를 만들고 확인한다.



print(indices)

# 인덱스 리스트를 랜덤하게 섞고 확인한다.



print(indices)

#### 3. 분할 비율(valid_size)에 따른 지점의 인덱스 값(split)을 계산한다.

In [ ]:
# 분할 비율(valid_size)에 해당하는 인덱스를 계산하고 확인해 본다.


print(split)

#### 4. split을 기준으로 학습 데이터 인덱스 리스트와 테스트 인덱스 리스트로 나눈다.

In [ ]:
# 학습 데이터 인덱스 리스트 및 테스트 인덱스 리스트를 만들고 확인해 본다.



print(train_idx)
print(test_idx)

#### 5. 데이터 세트들의 샘플러 및 로더를 만들고 확인한다.

In [ ]:
# 데이터 샘플링 방식(SubsetRandomSampler)을 지정한다
from torch.utils.data.sampler import SubsetRandomSampler



# 데이터 로딩을 위한 loader를 만든다. (sampler, 배치 사이즈 등 지정)



# 학습 loader와 테스트 loader의 class들을 출력하여 확인한다.
print(trainloader.dataset.classes)
print(testloader.dataset.classes)

### 코드들을 묶어서 load_split_train_test() 함수를 만든다.

In [ ]:
# 위의 코드들을 묶어서 load_split_train_test() 함수를 만든다. (입력 : 데이터 디렉토리, 분할 비율) (출력 : 학습 데이터 로더, 테스트 데이터 로더)

def load_split_train_test(data_dir, valid_size) :











    return trainloader, testloader

### load_split_train_test() 함수를 이용하여 trainloader, testloader를 생성한다.

In [ ]:
# load_split_train_test() 함수를 이용하여 trainloader와 testloader를 만들고 확인한다.




print(trainloader.dataset.classes)
print(testloader.dataset.classes)

## 이미지 데이터 샘플들을 살펴본다.

### 임의의 데이터를 로딩한 후 이미지와 레이블을 반환하는 get_random_images() 함수를 만든다.

In [ ]:
def get_random_images(num) :

    data = datasets.ImageFolder(data_dir, transform=t_transforms)





    from torch.utils.data.sampler import SubsetRandomSampler



    # loader에서 데이터를 한 개씩 꺼내 주는 iterator를 생성한다.



    return images, labels

### 임의 선택한 이미지를 표시해 본다.

In [ ]:
# 5개의 이미지와 레이블을 랜덤하게 가져온다.


# 픽셀 배열을 PIL 형식의 이미지로 변환하고 이미지 크기를 지정한다.
to_pil = transforms.ToPILImage()
fig = plt.figure(figsize=(20, 20))

# 학습 데이터의 class 리스트를 얻는다.



# 이미지를 표시하기 위한 설정을 한다.
for ii in range(len(images)) :
    image = to_pil(images[ii])
    sub = fig.add_subplot(1, len(images), ii+1)
    index = labels[ii].item()
    sub.set_title(classes[index])
    plt.axis('off')
    plt.imshow(image)

# 이미지를 표시한다.
plt.show()

# ResNet50 모델을 가져와 FCL(Fully Connected Layer)을 수정한다.

## Compute device를 정한다(CPU or GPU)

In [ ]:
# compute device를 정하고 확인한다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## 사전학습된 ResNet50 모델을 지정한다.

In [ ]:
# resnet50 모델을 pretrained=True로 설정한다.



### (확인) 수정 전의 ResNet50 모델을 확인해 본다. 

In [ ]:
print(model)

## FCL을 수정한다.(뉴런 구축, 신경망 연결, FCL의 layer 설정 등)

In [ ]:
# 모든 신경망 구축 : 전이학습을 위해 모델의 가중치를 freeze 한다.
for param in model.parameters():
    param.requires_grad = False

# 뉴런들을 연결하여 신경망을 생성한다.

model.fc = 




# 손실함수를 Cross entropy loss 함수로 지정한다.


# optimizer를 Adam으로 지정한다.


# 신경망을 compute device로 보낸다.
model.to(device)

# 종료 여부를 출력한다.
print('done!')

### (확인) FCL을 확인해 본다.

In [ ]:
print(model.fc)

# 모델의 FCL을 학습시키고 테스트 한다.

## 모델 학습/검증을 위한 변수를 설정한다.

In [ ]:
# 에폭 및 출력 간격을 설정한다.
epochs = 10
print_every = 5

# 손실 변수들을 초기화 한다.
running_loss = 0
train_losses, test_losses = [], []

# 현재의 학습 단계를 표현하는 steps 변수를 0으로 초기화 한다.
steps = 0

## 설정한 에폭만큼 모델을 학습시키며 검증/평가 한다.

In [ ]:
# 설정한 회수만큼 학습 후 검증한다.
for epoch in range(epochs) :
    # 에폭을 count 한다.
    epoch += 1
    # trainloader에서 이미지와 레이블을 하나씩 가져온다.
    for inputs, labels in trainloader:
        # 학습 단계를 count 하고 출력한다.
        steps += 1
        print('Training step ', steps)
        # 입력 데이터(이미지, 레이블)를 device로 이동시킨다.
        inputs, labels = inputs.to(device), labels.to(device)
        # 기존에 학습된 gradient 값을 초기화 한다.(이전에 학습한 값이 영향을 주지 않도록 함)
        optimizer.zero_grad()
        # 입력 데이터로 순전파를 수행하고 로그 확률을 얻는다.
        logps = model.forward(inputs)
        # 손실함수를 이용하여 손실(loss)값을 계산한다.
        loss = criterion(logps, labels)
        # 손실값을 기준으로 gradient를 update한다.
        loss.backward()
        # gradient를 이용하여 설정된 optimizer로 가중치를 update한다.
        optimizer.step()
        # 손실값을 누적/계산한다.
        running_loss += loss.item()
        # 학습 단계 5회마다 모델을 테스트/평가 한다.
        if steps % print_every == 0:
            # 손실과 정확도 변수를 초기화 한다.
            test_loss = 0
            accuracy = 0
            # 모델 평가 모드로 전환한다. (드롭아웃 등을 비활성화)
            model.eval()
            # 모델 평가 시 gradient를 계산하지 않도록 한다.
            with torch.no_grad():
                # testloader에서 이미지와 레이블을 하나씩 가져온다.
                for inputs, labels in testloader:
                    # 입력 데이터(이미지, 레이블)를 device로 이동시킨다.
                    inputs, labels = inputs.to(device), labels.to(device)
                    # 입력 데이터로 순전파를 수행하고 로그 확률을 얻는다.
                    logps = model.forward(inputs)
                    # 손실함수로 손실(loss) 값들을 계산한다.
                    batch_loss = criterion(logps, labels)
                    # 손실값을 누적시킨다.
                    test_loss += batch_loss.item()
                    # 로그 확률을 실제 확률값으로 변환한다.
                    ps = torch.exp(logps)
                    # 가장 큰 확률값과 class를 얻는다. (topk : 상위 k번째까지의 값)
                    top_p, top_class = ps.topk(1, dim=1)
                    # 정답 레이블들을 top_class와 동일한 형태로 바꾼 후 값을 비교하여, 맞춘 값들을 얻는다.
                    equals = top_class == labels.view(*top_class.shape)
                    # equals를 float 텐서로 바꾼 후 평균 정확도를 누적/계산한다.
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            # 학습 손실값과 테스트 손실값을 추가한다.
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            # 학습 손실값, 테스트 손실값, 테스트 정확도를 출력한다.
            print("Epoch {}/{}: ".format(epoch, epochs),
                  "Train loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test accuracy: {:.3f}\n".format(accuracy/len(testloader))) 
            # running_loss 값을 초기화 한다.
            running_loss = 0
            # 모델 학습 모드로 전환한다.
            model.train()
            break

### (확인) 학습 손실값과 테스트 손실값을 그래프로 확인한다.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

plt.plot(train_losses, label='training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

# 학습/테스트 완료된 모델을 저장한다.

In [ ]:
# 추후 로드하여 사용할 수 있도록 학습/테스트 완료된 모델을 저장한다.
torch.save(model, 'moonrockmodel.pth')

# 완성된 모델을 사용하여 예측한다.

## 저장한 모델을 불러온다.

In [ ]:
# 저장한 모델을 불러온다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load('moonrockmodel.pth')

### (확인) 불러온 모델을 확인해 본다.

In [ ]:
print(model)

## 이미지 예측을 위해 predict_image() 함수를 만든다.

In [ ]:
def predict_image(image):
    image_tensor = t_transforms(image).float()





    return index

## 5개의 이미지를 임의로 가져와 예측해 본다.

In [ ]:
# 모델 평가 모드로 전환한다.



# ToPILImage 변환을 이용하여 텐서 이미지를 PIL 이미지로 변경하는 변환 설정
to_pil = transforms.ToPILImage()

# 5개의 이미지를 랜덤하게 가져온다.



fig = plt.figure(figsize=(20, 20))

# 데이터의 class 목록을 얻는다.




# 5개의 이미지에 대해 반복 수행한다.
for ii in range(len(images)):
    # 이미지를 PIL 이미지로 변환한다.
    image = to_pil(images[ii])
    # 이미지에 대해 예측 후, 예측된 class를 얻는다.
    


    # 이미지 아래에 class를 표시하도록 설정한다.
    sub = fig.add_subplot(1, len(images), ii+1)
    res = labels[ii].item() == index
    sub.set_title(classes[index] + ':' + str(res))
    plt.axis('off')
    plt.imshow(image)

plt.show()